# WDTagger Playground

I just do some experiments on WDTagger and DeepDanbooru to see how they work and how much code they can share.

If I'm lucky enough, no modification is needed to make them work together.

In [22]:
from dbimutils import init_wdt
from deepbooru import init_deepbooru, get_deepbooru_tags_from_model

In [54]:
model, tags = init_wdt()
model_dd, tags_dd = init_deepbooru()
# model.summary()
# model_dd.summary()

In [55]:
from pprint import pprint
print("WDT w/h", model.input_shape[2], model.input_shape[1])
print("DD  w/h", model_dd.input_shape[2], model_dd.input_shape[1])
# https://github.com/KichangKim/DeepDanbooru/blob/c48689a85dde0e4a852c1691a7d746abe242e283/deepdanbooru/image/__init__.py#L25
# the resolution of WDT is 448 x 448 which is smaller

WDT w/h 448 448
DD  w/h 512 512


In [56]:
import random
pprint(random.sample(tags_dd, 5))
type(tags_dd) # so tags_dd is list[str]

['ginkgo_leaf', 'kettle', 'bullpup', 'mug', 'through_screen']


list

In [57]:
name = tags["name"].tolist()
pprint(random.sample(name, 5))

['1980s_(style)', 'blade', 'head_chain', 'branch', 'abyssal_ship']


In [58]:
from PIL import Image
image_path = "example.jpg"
image = Image.open(image_path).convert("RGB")

In [59]:
import dbimutils
import numpy as np
image_size = model.input_shape[1]
# okay...I will just copy and paste the code
img = dbimutils.smart_imread(image_path)
img = dbimutils.smart_24bit(img)
img = dbimutils.make_square(img, image_size)
img = dbimutils.smart_resize(img, image_size)
img = img.astype(np.float32)

probs = model.predict(np.array([img]))
tags["probs"] = probs[0]

thresh = 0.75

tags_names = tags[4:]
found_tags = tags_names[tags_names["probs"] > thresh][["tag_id", "name", "probs"]]
# result_tags_out = found_tags[["name", "probs"]].to_dict(orient="records")
result_tags_unsorted = found_tags["name"].to_list()
res = ", ".join(result_tags_unsorted)
pprint(res)
# pprint(found_tags)

1/1 [==============================] - 3s 3s/step
('1girl, solo, long_hair, hair_ornament, gloves, animal_ears, blue_hair, '
 'braid, white_hair, multicolored_hair, black_gloves, virtual_youtuber, '
 'rabbit_ears, twin_braids, two-tone_hair, animal_ear_fluff, pink_background, '
 'thick_eyebrows, food-themed_hair_ornament, fur-trimmed_gloves, '
 'carrot_hair_ornament')


In [60]:
threshold = 0.75
wdt_prompt = get_deepbooru_tags_from_model(model, name, image, 0.35)
dd_prompt = get_deepbooru_tags_from_model(model_dd, tags_dd, image, threshold)
pprint(wdt_prompt)
pprint(dd_prompt)
# okay... so wdt thinks it's monocrhome, that's interesting

1/1 [==============================] - 3s 3s/step
('monochrome, greyscale, solo, 1girl, lineart, transparent background, '
 'sensitive, long hair, looking at viewer')
('animal ears, rabbit ears, virtual youtuber, carrot hair ornament, '
 'food-themed hair ornament, animal ear fluff, multicolored hair, braid, hair '
 'ornament, rabbit girl, two-tone hair, 1girl, twin braids, blue hair, extra '
 'ears, white hair, playboy bunny, black gloves, black leotard, gloves, '
 'carrot, fur scarf, arm cuffs, leotard, fur-trimmed gloves, long hair, thick '
 'eyebrows, pink background, white sleeves, simple background, creature, '
 'eyebrows visible through hair, looking at viewer, white outline, fur trim, '
 'closed mouth')
